# Welcome to the GROMACS exercise for evaluating thermal conductivity of carbon nanotubes via NEMD method

Here follows the instructions for running a GROMACS simulation to compute thermal conductivity of single-wall carbon nanotubes. Provided by the Multi-Scale Modeling Lab of Politecnico di Torino (Italy).  These resources are intended for pedagogical purposes, and were designed for the undergraduate course: "Energy Applications of Materials", taught at Politecnico di Torino during spring 2026.
 
Matteo Fasano (matteo.fasano@polito.it)

Michele Pellegrino (michele.pellegrino@polito.it)

All rights reserved (2026)

In [ ]:
import nglview as ng
import MDAnalysis as mda

import numpy as np
import matplotlib.pyplot as plt
import os

from utilities_cnt import *

You can check if you correctly moved to the tutorial directory by running the command:

In [ ]:
!pwd

You can also check the content of the directory:

In [ ]:
!ls

Let's save the absolute path of the work directory:

In [ ]:
# Run this cell only once!
workdir = os.getcwd()
print("Work directory:",workdir)

## 1 - Setup preparation

First of all, what you need is a geometry file (file formats: `*.pdb`),  where all atom coordinates are indicated in the form of Cartesian coordinates.  In the `*.pdb` files also connections among atoms may be specified.  Readily available geometry generators can be found at: http://turin.nss.udel.edu/research/tubegenonline.html, or http://www.ugr.es/~gmdm/contub.htm.

We need first to convert the `*.pdb` geometry file (general and adopted also by many other software for MD) into `*.gro` (typical geometry file used by GROMACS). The following command can be used, where a periodic computational box of 16 nm x 16 nm x 16 nm is created, with the box edge (16 nm) being roughly 1.3 times the CNT length (periodicity is imposed along x, y and z): 

In [ ]:
!gmx editconf -f 0-INPUT/CNT53_12x057.pdb -o CNT53_12x057.gro -box 16 16 16

Next a topology (`*.top`) file needs to be created from the geometry file (`*.gro`). Topology files are basically a list of all the stretching, angular and dihedral bonds. In order to generate a `*.top` file from the corresponding `*.gro` file, the force-field files are requested. In particular, for this step the two required files are: `ffoplsaaCNT.n2t` and `ffoplsaaCNT.rtp` are used (included in the `ffoplsaaCNT.ff` subfolder).

Enter into the 0-INPUT folder:

In [ ]:
%cd {workdir}/0-INPUT

and then run the `x2top` command:

In [ ]:
!gmx x2top -f ../CNT53_12x057.gro -o CNT53_12x057.top -ff ffoplsaaCNT -noparam

Check the content of the created topology file:

In [ ]:
!cat CNT53_12x057.top

Finally, let us go back to the main folder:

In [ ]:
%cd {workdir}

## 2 - Energy minimization

Once the correct `*.top` file is available, we are ready to perform MD simulations. Before starting, the energy minimization of the simulated structure needs to be performed. In this case it is not strictly required, because our CNT is already in the configuration of minimal energy. However, in general, energy minimization is always necessary for a correct numerical convergence of the subsequent steps. Here, we need an `*.mdp` file where prescriptions on the numerical simulations are indicated (an `*.mdp` file is already included in the subfolder 1-EM). First, we pre-compile di simulation:

In [ ]:
!gmx grompp -f 1-EM/em.mdp -c CNT53_12x057.gro -p ./0-INPUT/CNT53_12x057.top -o 1-EM/CNT53_12x057_em.tpr -po 1-EM/mdout.mdp

then, we run it:

In [ ]:
!gmx mdrun -s 1-EM/CNT53_12x057_em.tpr -o 1-EM/CNT53_12x057_em.trr -c CNT53_12x057_em.gro -g 1-EM/em.log -e 1-EM/em.edr -v

Since GROMACS discontinued the basic visualization tool `gmx view`, we are going to use a `nglview` widget instead:

In [ ]:
u_cnt = mda.Universe('1-EM/CNT53_12x057_em.tpr', '1-EM/CNT53_12x057_em.trr')
view_cnt = ng.show_mdanalysis(u_cnt)
view_cnt

However, a much better visualization (of many geometry files such as: `*.pdb`, `*.gro`, `*.xyz`, etc., as well as of trajectories: `*.trr`) can be obtained by VMD (https://www.ks.uiuc.edu/Research/vmd/), VEGA-ZZ (http://www.vegazz.net/) or OVITO (https://www.ovito.org/). For downloading them, you just need to make a registration. `*.pdb` files can be visualized also by `molviewer` readily available in MATLAB.

In [ ]:
# If you have installed VMD:
!vmd 1-EM/CNT53_12x057_em.trr CNT53_12x057_em.gro

## 3 - Thermal equilibration

Here, we want to thermalize the whole system by attaching a Stochastic Velocity Rescale thermostat to the CNT. Therefore, we need an `*.mdp` file where prescriptions on the numerical simulations are indicated. An `*.mdp` file is already included in the subfolder 2-TE, let's have a look:

In [ ]:
!cat 2-TE/*.mdp

The command `grompp` performs a preparation step before running the real md simulation and it produces a `*.tpr` file. In order to start the simulation from a previous minimal energy state (Energy minimization), use here the flag `-t` as follows:

In [ ]:
!gmx grompp -f 2-TE/md-te.mdp -c CNT53_12x057_em.gro -p ./0-INPUT/CNT53_12x057.top -t 1-EM/CNT53_12x057_em.trr -o 2-TE/CNT53_12x057_md-te.tpr -po 2-TE/mdout.mdp

The real md simulation is performed by the command `mdrun`, which accepts the `*.tpr` as an input and it produces a `*.trr` file as an output, as follows:

In [ ]:
!gmx mdrun -s 2-TE/CNT53_12x057_md-te.tpr -o 2-TE/CNT53_12x057_md-te.trr -c CNT53_12x057_md-te.gro -g 2-TE/md-te.log -e 2-TE/md-te.edr -v 

**Tip** - If you want to repeat the simulation, but run a different number of steps, there's no need to recompile, just use the `-nsteps` flag:
    
    gmx mdrun -s 2-TE/CNT53_12x057_md-te.tpr ... -v -nsteps <possibly-different-number-of-steps>
    
So you can run a longer or a shorter simulation (depending on how powerful your laptop is).

After that the simulation is finished, we can extract the temperature/energy of the system from the trajectory using `gmx energy`. Since we are not using an interactive shell, the desired observable (`Temperature` in this case) is going to be passed to the function using `echo`:

In [ ]:
!echo Temperature | gmx energy -f 2-TE/md-te.edr -o 2-TE/energies.xvg

For plotting temperature/energy history one can directly use `xmgrace`, if installed:

In [ ]:
# If you have installed xmgrace:
!xmgrace -free -nxy 2-TE/energies.xvg

Otherwise `.xvg` can be read and plotted using a combination of `numpy` and `matplotlib`:

In [ ]:
temperature = np.loadtxt("2-TE/energies.xvg", comments=("@", "#"))
fig1, ax1 = plt.subplots()
plt.plot(temperature[:,0],temperature[:,1],'r-')
plt.xlabel('time [ps]')
plt.ylabel('temperature [K]')

Again, the simulated trajectory can be opened with `nglview`:

In [ ]:
u_cnt = mda.Universe('2-TE/CNT53_12x057_md-te.tpr', '2-TE/CNT53_12x057_md-te.trr')
view_cnt = ng.show_mdanalysis(u_cnt)
view_cnt

## 3 - Non-equilibirum simulation (NEMD)

Here, for computing the thermal conductivity of the carbon nanotube we perform a non-equilibrium computation, where the one end of the tube is attached to a Nose-Hoover (NH) thermostat at 320 K, the other end is attached to a NH thermostat at 280 K. Thermal conductivity is then computed by measuring both the slope of the temperature profile and the heat flux. 

For this computation, we need to subdivide all carbon atoms into several groups: this is done by creating an index file `*.ndx` (here, you can find it in the subfolder `0-INPUT`). However, a Python function (`make_index()`) is included in the library `utilities_cnt.py` for making your own groups (instructions are therein included).

In [ ]:
# This is just an example, find the actual *.ndx file in 0-INPUT/
IV = 1
IN = 784//12
GN = 12
make_index(IV, IN, GN)

As above, we need an `*.mdp` file where all prescriptions/inputs on the numerical simulations are provided (an `*.mdp` file is already included in the subfolder `3-NE`). Detailed instructions need to be given in the `*.mdp` file for specifying thermostats for each group of atoms. The provided Python function `make_nose_hoover()` (included in the subfolder `utilities_cnt.py`) will enable  you to produce the proper text to be included in the `*.mdp` file before performing the `grompp` command. In this case, we have already done this step and the provided `*.mdp` file is consistent.

In [ ]:
# This is just an example, find the actual *.mdp file in 3-NE/
NN = 1
TI = 280
TF = 320
tau = 0.2
make_nose_hoover(GN, NN, TI, TF, tau)

You can find a MATLAB equivalent to `make_index()` and `make_nose_hoover()` in `_POST-PROCESSING/MATLAB_VERSION`. 

Let's inspect the `*.mdp` file:

In [ ]:
!cat 3-NE/md-ne.mdp

The command `grompp` performs a preparation step before running the real md simulation and it produces a `*.tpr` file. In order to start the simulation from a the previous equilibrium state (equilibration) use here the flag `-t`:

In [ ]:
!gmx grompp -n 0-INPUT/CNT53-long.ndx -f 3-NE/md-ne.mdp -c CNT53_12x057_md-te.gro -t 2-TE/CNT53_12x057_md-te.trr -p ./0-INPUT/CNT53_12x057.top -o 3-NE/CNT53_12x057_md-ne.tpr -po 3-NE/mdout.mdp -maxwarn 1

**Tip** - Try to remove the `-maxwarn 1` flag and see what happens: GROMACS will throw a warning since some temperature coupling groups do not use temperature coupling. This is perfectly ok in our case, since we are doing NEMD, but mind that in general Gromacs will throw warning in **many** occasions. So always read warnings and decide if they are innocuous or not!

The real md simulation is performed by the command mdrun, which accepts the `*.tpr` as an input and it produces a `*.trr` file as an output, as follows:

In [ ]:
!gmx mdrun -s 3-NE/CNT53_12x057_md-ne.tpr -o 3-NE/CNT53_12x057_md-ne.trr -c CNT53_12x057_md-ne.gro -g 3-NE/md-ne.log -e 3-NE/md-ne.edr

The run progess is shown in the `*.log` file:

In [ ]:
!cat 3-NE/*log

Checking the `*.log` file may be useful when running a simulation on the background or in a queue.

**Tip** - At the end of the `*.log` file, GROMACS reports statistics on computational efficiency. You can get an estimate of how fast is the simulation (ns/day and real-hour/ns) and which activities are consuming the most time. This information is very useful if you are running becnhmarks or just trying to make your simulation more efficient.

For extracting the temperature/energy history use `gmx energy` and then type the numbers corresponding to the quantities to extract, as suggested by the interactive menu. In this case type from 35 to 86 (and we provide them by `echo` command), in order to analyze the trend of temperature in the different groups of carbon atoms of the nanotube at steady state conditions, namely after 50 ps:

In [ ]:
!echo "35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 0" | gmx energy -f 3-NE/md-ne.edr -o _POST-PROCESSING/Temp.xvg -b 50 -e 200

In [ ]:
!cat _POST-PROCESSING/Temp.xvg

The heat flux through the nanotube can be computed as:

$q = f \cdot N_{DOF} \cdot k_B \cdot T_a / S$,

being:

$f$ : Nose-Hoover (NH) friction factor [s$^{-1}$]

$N_{DOF}$ : Number of degrees of freedom attached to NH [-]

$k_B$ : Boltzmann constant [J/K]

$T_a$ : Average temperature of NH [K]

$S$ : Tube cross-section surface [m$^2$]

See the slides for a more comprehensive explanation. The average **friction factor** during the simulation at steady state conditions can be extracted as:

In [ ]:
!echo "vXi-INT52" | gmx energy -f 3-NE/md-ne.edr -o _POST-PROCESSING/FrictionNH.xvg -b 50 -e 200

Finally, the post-processing (computation of thermal conductivity) can be performed by the function `compute_lambda()`, also included in the `utilities_cnt.py`. The average value of friction factor of the NH thermostat (group `INT52`) should be then passed to the function. 

In [ ]:
l=12
n=52
ne=2
file="_POST-PROCESSING/Temp.xvg"
xi = ### Here goes the value you got from MD (NB in 1/s, so remember to convert!) ##
Tmean, Tfit, q = compute_lambda(l, n, ne, file, xi)

In [ ]:
# Linear fit: f(x) = a*x + b
a, b = np.polyfit(Tfit[:, 0], Tfit[:, 1], 1)

# Thermal conductivity (now in W/mK)
k = q / (a * 1e9)

# Fit line (same range as [0:12])
x_line = np.linspace(0.0, 12.0, 200)
y_line = a * x_line + b

plt.figure()
plt.plot(Tmean[:, 0], Tmean[:, 1], 'ro', label='MD')
plt.plot(x_line, y_line, 'b-', label='fit')
plt.xlabel('CNT length [nm]')
plt.ylabel('Mean Temperature [K]')
plt.title(f'k = {k:.2f} [W/mK]')
plt.legend()

plt.show()

## Final remarks

...